# Non-Circular λ₁ Test: Sweep L on TCS K7

**Goal**: Test if λ₁(L) ~ c/L² and determine c WITHOUT assuming L = 8.354

## Key Question
Is the coefficient c = π² (from neck mode) or something else?

## Method
1. Build SPD-guaranteed metric for various L
2. Discretize Laplacian on K7 mesh
3. Compute λ₁ numerically
4. Fit λ₁(L) = c/L² and extract c

In [ ]:
# Colab setup
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Use GPU if available
    try:
        import cupy as cp
        from cupyx.scipy.sparse import csr_matrix as cp_csr
        from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
        USE_GPU = True
        print("Using CuPy (GPU)")
    except:
        USE_GPU = False
        print("CuPy not available, using NumPy (CPU)")
else:
    USE_GPU = False

import numpy as np
from scipy.sparse import diags, csr_matrix
from scipy.sparse.linalg import eigsh
from scipy.linalg import expm, logm
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, List
import json
import time

np.random.seed(42)
print(f"NumPy version: {np.__version__}")

## 1. SPD-Guaranteed Metric Construction

Use Cholesky parametrization: g = L @ L.T + εI

In [ ]:
def cholesky_to_metric(L_chol: np.ndarray, eps: float = 1e-8) -> np.ndarray:
    """Convert lower triangular L to SPD metric g = L @ L.T + eps*I"""
    return L_chol @ L_chol.T + eps * np.eye(L_chol.shape[0])

def metric_to_cholesky(g: np.ndarray) -> np.ndarray:
    """Extract Cholesky factor from SPD metric"""
    return np.linalg.cholesky(g)

def log_euclidean_interp(g1: np.ndarray, g2: np.ndarray, t: float) -> np.ndarray:
    """
    Log-Euclidean interpolation between SPD matrices.
    g(t) = exp((1-t)*log(g1) + t*log(g2))
    
    Guarantees SPD output for any t ∈ [0,1].
    """
    log_g1 = logm(g1)
    log_g2 = logm(g2)
    return expm((1 - t) * log_g1 + t * log_g2)

print("SPD utilities loaded")

## 2. TCS Metric with Variable L

In [ ]:
@dataclass
class TCSMetric:
    """
    TCS G₂ metric on K7 with variable neck length L.
    
    Uses SPD-safe construction throughout.
    """
    L: float  # Neck length (VARIABLE - this is what we sweep)
    delta: float = 2.0  # Gluing width
    
    def smooth_step(self, x: float) -> float:
        """C^∞ smooth step from 0 to 1"""
        if x <= 0:
            return 0.0
        if x >= 1:
            return 1.0
        return x**3 * (10 - 15*x + 6*x**2)
    
    def base_metric_neck(self) -> np.ndarray:
        """Base 7×7 metric in neck region (product structure)"""
        # g = dt² + g_K3 + dθ² + dψ²
        # For K3, use flat T⁴ approximation
        return np.eye(7)
    
    def base_metric_end(self, side: str) -> np.ndarray:
        """Base metric at compact ends (slight deformation)"""
        g = np.eye(7)
        # Add small curvature correction
        if side == 'left':
            g[0, 0] = 1.1  # dt² slightly larger
            g[5, 5] = 1.05  # dθ² 
        else:
            g[0, 0] = 1.1
            g[6, 6] = 1.05  # dψ²
        return g
    
    def metric_at_t(self, t: float) -> np.ndarray:
        """
        7×7 SPD metric at neck parameter t.
        
        Uses log-Euclidean interpolation for SPD guarantee.
        """
        # Cutoff parameter
        x = (t + self.L) / (2 * self.L)  # 0 at t=-L, 1 at t=L
        chi = self.smooth_step(x)
        
        # Base metrics
        g_left = self.base_metric_end('left')
        g_right = self.base_metric_end('right')
        g_neck = self.base_metric_neck()
        
        # Three-way interpolation via log-Euclidean
        # Near left (chi~0): g_left
        # Near right (chi~1): g_right  
        # In middle: g_neck
        
        # Weight for neck region
        w_neck = 4 * chi * (1 - chi)  # Peaks at chi=0.5
        w_left = (1 - chi) * (1 - w_neck)
        w_right = chi * (1 - w_neck)
        
        # Normalize
        total = w_left + w_neck + w_right
        w_left /= total
        w_neck /= total
        w_right /= total
        
        # Log-Euclidean weighted mean
        log_g = (w_left * logm(g_left) + 
                 w_neck * logm(g_neck) + 
                 w_right * logm(g_right))
        
        return expm(log_g)
    
    def verify_spd(self, n_samples: int = 50) -> dict:
        """Verify metric is SPD everywhere"""
        t_vals = np.linspace(-self.L * 0.95, self.L * 0.95, n_samples)
        min_eigs = []
        dets = []
        
        for t in t_vals:
            g = self.metric_at_t(t)
            eigvals = np.linalg.eigvalsh(g)
            min_eigs.append(np.min(eigvals))
            dets.append(np.linalg.det(g))
        
        return {
            'all_positive': all(e > 0 for e in min_eigs),
            'min_eigenvalue': min(min_eigs),
            'det_range': (min(dets), max(dets))
        }

# Test
test_metric = TCSMetric(L=8.0)
result = test_metric.verify_spd()
print(f"SPD verification: {result}")

## 3. Discrete Laplacian on TCS

We use a simplified 1D model along the neck direction, which captures the dominant λ₁ ~ 1/L² behavior.

In [ ]:
def build_laplacian_1d(metric: TCSMetric, N: int = 100) -> csr_matrix:
    """
    Build 1D Laplacian along neck direction.
    
    Δf = (1/√g) ∂_t(√g g^{tt} ∂_t f)
    
    For our metric with g_tt ~ 1, this simplifies.
    """
    L = metric.L
    h = 2 * L / (N - 1)  # Grid spacing
    t_vals = np.linspace(-L, L, N)
    
    # Get metric at each point
    sqrt_det_g = np.zeros(N)
    g_tt_inv = np.zeros(N)  # g^{tt}
    
    for i, t in enumerate(t_vals):
        g = metric.metric_at_t(t)
        sqrt_det_g[i] = np.sqrt(np.linalg.det(g))
        g_inv = np.linalg.inv(g)
        g_tt_inv[i] = g_inv[0, 0]  # t is index 0
    
    # Build sparse Laplacian matrix
    # Δf_i ≈ (1/√g_i) * [(√g g^tt)_{i+1/2} (f_{i+1} - f_i) - (√g g^tt)_{i-1/2} (f_i - f_{i-1})] / h²
    
    diag_main = np.zeros(N)
    diag_upper = np.zeros(N - 1)
    diag_lower = np.zeros(N - 1)
    
    for i in range(1, N - 1):
        # Coefficients at half-points
        coeff_plus = 0.5 * (sqrt_det_g[i] * g_tt_inv[i] + sqrt_det_g[i+1] * g_tt_inv[i+1])
        coeff_minus = 0.5 * (sqrt_det_g[i] * g_tt_inv[i] + sqrt_det_g[i-1] * g_tt_inv[i-1])
        
        diag_main[i] = -(coeff_plus + coeff_minus) / (h**2 * sqrt_det_g[i])
        diag_upper[i] = coeff_plus / (h**2 * sqrt_det_g[i])
        diag_lower[i-1] = coeff_minus / (h**2 * sqrt_det_g[i])
    
    # Boundary conditions: Neumann (∂f/∂n = 0)
    # This makes λ₀ = 0 (constant mode)
    diag_main[0] = -1 / h**2
    diag_upper[0] = 1 / h**2
    diag_main[-1] = -1 / h**2
    diag_lower[-1] = 1 / h**2
    
    # Build sparse matrix
    Delta = diags([diag_lower, diag_main, diag_upper], [-1, 0, 1], format='csr')
    
    return -Delta  # Return positive semi-definite operator

# Test
test_Delta = build_laplacian_1d(test_metric, N=100)
print(f"Laplacian shape: {test_Delta.shape}")
print(f"Laplacian is sparse: {test_Delta.nnz} non-zeros")

## 4. Compute λ₁ for Given L

In [ ]:
def compute_lambda1(L: float, N: int = 200) -> Tuple[float, np.ndarray]:
    """
    Compute first non-zero eigenvalue λ₁ for TCS with neck length L.
    
    Returns (λ₁, eigenfunction)
    """
    metric = TCSMetric(L=L)
    Delta = build_laplacian_1d(metric, N=N)
    
    # Find smallest eigenvalues
    # λ₀ = 0 (constant), λ₁ is what we want
    try:
        # Use 'SA' (smallest algebraic) for positive semi-definite
        eigenvalues, eigenvectors = eigsh(Delta, k=3, which='SA')
        
        # Sort and get λ₁ (skip λ₀ ≈ 0)
        idx = np.argsort(eigenvalues)
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        
        # λ₀ should be ~0, λ₁ is the spectral gap
        lambda1 = eigenvalues[1] if eigenvalues[0] < 1e-6 else eigenvalues[0]
        psi1 = eigenvectors[:, 1] if eigenvalues[0] < 1e-6 else eigenvectors[:, 0]
        
        return lambda1, psi1
    
    except Exception as e:
        print(f"Eigenvalue computation failed for L={L}: {e}")
        return np.nan, np.zeros(N)

# Test at L = 8
lambda1_test, psi1_test = compute_lambda1(L=8.0, N=200)
print(f"λ₁(L=8) = {lambda1_test:.6f}")
print(f"Expected π²/L² = {np.pi**2 / 64:.6f}")

## 5. MAIN: Sweep L and Fit λ₁(L) = c/L²

In [ ]:
# Sweep L values
L_values = np.array([5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 20])
lambda1_values = []

print("Computing λ₁(L) for various L...")
print(f"{'L':>6} {'λ₁':>12} {'π²/L²':>12} {'λ₁·L²':>12}")
print("-" * 45)

for L in L_values:
    start = time.time()
    lambda1, _ = compute_lambda1(L, N=300)
    elapsed = time.time() - start
    lambda1_values.append(lambda1)
    
    pi_sq_over_L2 = np.pi**2 / L**2
    lambda1_times_L2 = lambda1 * L**2
    
    print(f"{L:>6.1f} {lambda1:>12.6f} {pi_sq_over_L2:>12.6f} {lambda1_times_L2:>12.4f}")

lambda1_values = np.array(lambda1_values)

In [ ]:
# Fit λ₁ = c / L²
# Taking log: log(λ₁) = log(c) - 2*log(L)
# Or equivalently: λ₁ * L² = c (constant)

# Method 1: Direct average of λ₁ * L²
c_direct = np.mean(lambda1_values * L_values**2)
c_direct_std = np.std(lambda1_values * L_values**2)

# Method 2: Linear regression on log-log scale
log_L = np.log(L_values)
log_lambda = np.log(lambda1_values)
slope, intercept = np.polyfit(log_L, log_lambda, 1)
c_fit = np.exp(intercept)

print("\n" + "=" * 50)
print("RESULTS: λ₁(L) = c / L^α")
print("=" * 50)
print(f"\nMethod 1 (direct): c = {c_direct:.4f} ± {c_direct_std:.4f}")
print(f"Method 2 (log-log fit): c = {c_fit:.4f}, α = {-slope:.4f}")
print(f"\nReference values:")
print(f"  π² = {np.pi**2:.4f}")
print(f"  14 (dim G₂) = 14.0000")
print(f"\nRatio c/π² = {c_direct / np.pi**2:.4f}")
print(f"Ratio c/14 = {c_direct / 14:.4f}")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: λ₁ vs L
ax1 = axes[0]
ax1.scatter(L_values, lambda1_values, s=50, c='blue', label='Computed')
L_fine = np.linspace(min(L_values), max(L_values), 100)
ax1.plot(L_fine, c_direct / L_fine**2, 'r-', label=f'Fit: c/L², c={c_direct:.2f}')
ax1.plot(L_fine, np.pi**2 / L_fine**2, 'g--', label=f'π²/L²')
ax1.set_xlabel('L (neck length)')
ax1.set_ylabel('λ₁')
ax1.set_title('Spectral Gap vs Neck Length')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: λ₁ * L² (should be constant = c)
ax2 = axes[1]
ax2.scatter(L_values, lambda1_values * L_values**2, s=50, c='blue')
ax2.axhline(y=c_direct, color='r', linestyle='-', label=f'Mean: {c_direct:.2f}')
ax2.axhline(y=np.pi**2, color='g', linestyle='--', label=f'π² = {np.pi**2:.2f}')
ax2.axhline(y=14, color='orange', linestyle=':', label='14 (dim G₂)')
ax2.set_xlabel('L')
ax2.set_ylabel('λ₁ · L²')
ax2.set_title('Coefficient c = λ₁ · L²')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Log-log
ax3 = axes[2]
ax3.scatter(np.log(L_values), np.log(lambda1_values), s=50, c='blue', label='Data')
ax3.plot(log_L, slope * log_L + intercept, 'r-', label=f'Fit: slope = {slope:.3f}')
ax3.set_xlabel('log(L)')
ax3.set_ylabel('log(λ₁)')
ax3.set_title(f'Log-Log Plot (slope should be -2)')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('lambda1_sweep_results.png', dpi=150)
plt.show()

print("\nPlot saved to lambda1_sweep_results.png")

## 6. Key Test: What L Gives λ₁ = 14/99?

In [ ]:
# If λ₁ = c/L², and we want λ₁ = 14/99, then:
# L² = c * 99/14
# L = √(c * 99/14)

target_lambda1 = 14 / 99

# Using our measured c
L_for_gift = np.sqrt(c_direct * 99 / 14)

# Using c = π² (theoretical)
L_theoretical = np.sqrt(np.pi**2 * 99 / 14)

print("\n" + "=" * 50)
print("TEST: What L gives λ₁ = 14/99?")
print("=" * 50)
print(f"\nTarget: λ₁ = 14/99 = {target_lambda1:.6f}")
print(f"\nUsing measured c = {c_direct:.4f}:")
print(f"  L* = √(c × 99/14) = {L_for_gift:.4f}")
print(f"\nUsing theoretical c = π²:")
print(f"  L* = √(π² × 99/14) = {L_theoretical:.4f}")
print(f"\nGIFT canonical: L = 8.354")
print(f"\nκ = L*²/H* comparison:")
print(f"  Measured: κ = {L_for_gift**2 / 99:.4f}")
print(f"  Theoretical: κ = π²/14 = {np.pi**2 / 14:.4f}")

## 7. Summary and Export

In [ ]:
# Compile results
results = {
    'test_description': 'Non-circular sweep of L to measure c in λ₁ = c/L²',
    'L_values': L_values.tolist(),
    'lambda1_values': lambda1_values.tolist(),
    'lambda1_times_L2': (lambda1_values * L_values**2).tolist(),
    'fit_results': {
        'c_direct_mean': float(c_direct),
        'c_direct_std': float(c_direct_std),
        'c_loglog_fit': float(c_fit),
        'slope_loglog': float(slope),
        'expected_slope': -2.0
    },
    'reference_values': {
        'pi_squared': float(np.pi**2),
        'dim_G2': 14,
        'ratio_c_over_pi2': float(c_direct / np.pi**2),
        'ratio_c_over_14': float(c_direct / 14)
    },
    'gift_test': {
        'target_lambda1': float(target_lambda1),
        'L_for_target_measured': float(L_for_gift),
        'L_for_target_theoretical': float(L_theoretical),
        'L_gift_canonical': 8.354,
        'kappa_measured': float(L_for_gift**2 / 99),
        'kappa_theoretical': float(np.pi**2 / 14)
    },
    'conclusion': {
        'c_is_pi_squared': abs(c_direct - np.pi**2) < 0.5,
        'slope_is_minus_2': abs(slope + 2) < 0.1,
        'kappa_matches_prediction': abs(L_for_gift**2 / 99 - np.pi**2 / 14) < 0.05
    }
}

# Save
with open('lambda1_sweep_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n" + "=" * 60)
print("FINAL CONCLUSIONS")
print("=" * 60)
print(f"\n1. λ₁(L) scales as: λ₁ ∝ 1/L^{-slope:.2f}")
print(f"   Expected: λ₁ ∝ 1/L² (slope = -2.00)")
print(f"   Status: {'✓ CONFIRMED' if abs(slope + 2) < 0.2 else '✗ UNEXPECTED'}")
print(f"\n2. Coefficient c = {c_direct:.4f}")
print(f"   Reference π² = {np.pi**2:.4f}")
print(f"   Status: {'✓ MATCHES π²' if abs(c_direct - np.pi**2) < 1 else '✗ DIFFERS FROM π²'}")
print(f"\n3. κ = L²/H* for λ₁ = 14/99:")
print(f"   Measured: {L_for_gift**2 / 99:.4f}")
print(f"   GIFT prediction (π²/14): {np.pi**2 / 14:.4f}")
print(f"   Status: {'✓ MATCHES' if abs(L_for_gift**2 / 99 - np.pi**2 / 14) < 0.1 else '✗ DIFFERS'}")
print("\nResults saved to lambda1_sweep_results.json")
print("=" * 60)

## 8. Diagnostic: Eigenfunction Visualization

In [ ]:
# Plot eigenfunction for a few L values
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

L_show = [6, 8, 10, 14]

for ax, L in zip(axes.flat, L_show):
    metric = TCSMetric(L=L)
    N = 200
    t_vals = np.linspace(-L, L, N)
    
    lambda1, psi1 = compute_lambda1(L, N=N)
    
    # Normalize eigenfunction
    psi1 = psi1 / np.max(np.abs(psi1))
    
    # Compare with sin(πt/L)
    sin_mode = np.sin(np.pi * (t_vals + L) / (2 * L))
    sin_mode = sin_mode / np.max(np.abs(sin_mode))
    
    ax.plot(t_vals, psi1, 'b-', linewidth=2, label='Computed ψ₁')
    ax.plot(t_vals, sin_mode, 'r--', linewidth=1, label='sin(πt/L) reference')
    ax.set_xlabel('t (neck parameter)')
    ax.set_ylabel('ψ₁(t)')
    ax.set_title(f'L = {L}, λ₁ = {lambda1:.4f}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.savefig('eigenfunctions.png', dpi=150)
plt.show()

print("Eigenfunction plot saved to eigenfunctions.png")